# ART RECOMMENDER SYSTEM

## Importing and Cleaning Data - MET

This script will retrieve information from the official [MET GitHub](https://github.com/metmuseum), use it to scrape the description from everal works of art in their [Webpage](https://www.metmuseum.org/) and save it in a .csv file that will be used in the rest of the analysis to avoid running all these steps all over again.

This script should be executed just **one time**.  To repeat the rest of the analysis, if necessary, it can just be executed starting on next step ([Data Analysis for MET](03_MET_AnalysisTFIDF.ipynb))

In [1]:
import pandas as pd
import requests
import pickle
import time
import re
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup

**Loading the files can be done directly through the GutHub page...**

In [2]:
met_df = pd.read_csv('https://github.com/metmuseum/openaccess/raw/master/MetObjects.csv')
len(met_df)

C:\Users\Asus\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7,8,9,10,11,18,27,28,29,30,31,32,33,34,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


472255

**... Or by downloading the :CSV file locally first**

In [3]:
met_df = pd.read_csv('../files/MetObjects.csv')
len(met_df)

/Users/monita/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,8,9,10,11,18,27,28,29,30,31,32,33,34,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


466142

In [4]:
met_df.columns

Index(['Object Number', 'Is Highlight', 'Is Public Domain', 'Object ID',
       'Department', 'Object Name', 'Title', 'Culture', 'Period', 'Dynasty',
       'Reign', 'Portfolio', 'Artist Role', 'Artist Prefix',
       'Artist Display Name', 'Artist Display Bio', 'Artist Suffix',
       'Artist Alpha Sort', 'Artist Nationality', 'Artist Begin Date',
       'Artist End Date', 'Object Date', 'Object Begin Date',
       'Object End Date', 'Medium', 'Dimensions', 'Credit Line',
       'Geography Type', 'City', 'State', 'County', 'Country', 'Region',
       'Subregion', 'Locale', 'Locus', 'Excavation', 'River', 'Classification',
       'Rights and Reproduction', 'Link Resource', 'Metadata Date',
       'Repository'],
      dtype='object')

**How many works are classified as Paintings?**

In [6]:
len(met_df[met_df['Object Name']=='Painting'])

6070

**What oher types of art are there in the collection?**

In [5]:
met_df.groupby('Object Name')['Object Number'].count()

Object Name
\r\n                                                     4
\r\nAmulet                                               1
\r\nDress                                                1
\r\nFragment, knife                                      1
\r\nPulley                                               1
\r\nSpool                                                1
\r\nT-shirt                                              1
 Ewer                                                    1
 Vase                                                    4
"Autophone" Organette                                    1
"Basso"                                                  1
"Humantone" Nose Flute                                   1
"Japanese Fiddle"                                        1
"Komo" Trumpet                                           1
"Ladies in Blue" Fresco                                  1
"Musical Ball" ("Return Ball")                           1
"New Year's Bottle" with cow goddess Nebethe

In [18]:
met_df.head()

,Object Number,Is Highlight,Is Public Domain,Object ID,Department,Object Name,Title,Culture,Period,Dynasty,...,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Metadata Date,Repository
0,1979.486.1,False,False,1,American Decorative Arts,Coin,One-dollar Liberty Head Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/1,5/21/2018 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
1,1980.264.5,False,False,2,American Decorative Arts,Coin,Ten-dollar Liberty Head Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/2,5/21/2018 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
2,67.265.9,False,False,3,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/3,5/21/2018 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
3,67.265.10,False,False,4,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/4,5/21/2018 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
4,67.265.11,False,False,5,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/5,5/21/2018 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"


In [5]:
met_df['Object Name'].head()

0    Coin
1    Coin
2    Coin
3    Coin
4    Coin
Name: Object Name, dtype: object

In [6]:
met_df['Title'].head()

0    One-dollar Liberty Head Coin
1    Ten-dollar Liberty Head Coin
2      Two-and-a-Half Dollar Coin
3      Two-and-a-Half Dollar Coin
4      Two-and-a-Half Dollar Coin
Name: Title, dtype: object

In [7]:
met_df['Credit Line'].head()

0    Gift of Heinz L. Stoppelmann, 1979
1    Gift of Heinz L. Stoppelmann, 1980
2     Gift of C. Ruxton Love, Jr., 1967
3     Gift of C. Ruxton Love, Jr., 1967
4     Gift of C. Ruxton Love, Jr., 1967
Name: Credit Line, dtype: object

In [3]:
met_df[met_df['Link Resource'] == 'http://www.metmuseum.org/art/collection/search/437216'].values.tolist()

[['1987.290.3a–p',
  False,
  True,
  437216,
  'European Paintings',
  'Painting',
  'The Fifteen Mysteries and the Virgin of the Rosary',
  nan,
  nan,
  nan,
  nan,
  nan,
  'Artist',
  nan,
  'Netherlandish Painter (possibly Goswijn van der Weyden, active by 1491, died after 1538), ca. 1515–20',
  nan,
  nan,
  'Netherlandish Painter (possibly Goswijn van der Weyden',
  'Netherlandish',
  '1491      ',
  '1538      ',
  nan,
  1515,
  1520,
  'Oil on wood',
  '(a) 9 7/8 x 21 in. (25.1 x 53.3 cm); (b–p) each 5 x 4 1/8 in. (12.7 x 10.5 cm)',
  'Anonymous Bequest, 1984',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  'Paintings',
  nan,
  'http://www.metmuseum.org/art/collection/search/437216',
  '5/21/2018 8:00:02 AM',
  'Metropolitan Museum of Art, New York, NY']]

### Data Scraping

In [53]:
requests.get("https://www.metmuseum.org/art/collection/search/435845").text

'<html style="height:100%"><head><META NAME="ROBOTS" CONTENT="NOINDEX, NOFOLLOW"><meta name="format-detection" content="telephone=no"><meta name="viewport" content="initial-scale=1.0"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"></head><body style="margin:0px;height:100%"><iframe src="/_Incapsula_Resource?CWUDNSAI=28&xinfo=9-7381157-0%200NNN%20RT%281527010513622%2068%29%20q%280%20-1%20-1%20-1%29%20r%280%20-1%29%20B12%284%2c315%2c0%29%20U2&incident_id=701009300106960731-43383666566314537&edet=12&cinfo=04000000" frameborder=0 width="100%" height="100%" marginheight="0px" marginwidth="0px">Request unsuccessful. Incapsula incident ID: 701009300106960731-43383666566314537</iframe></body></html>'

**Test for one webpage**

In [52]:
url_list = ['http://www.metmuseum.org/art/collection/search/435845']
full_list = []
c = 0
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
cookies = {
     'incap_ses_415_965359':'djRha9OqhshstDcXvPV8cmHCBQGBKloAAAAAN3/D9dvoqwEc7GPEwefkhQ==', 'PHPSESSID':'fjmr7plc0dmocm8roq7togcp92', 'visid_incap_965359':'akteT8lDT1iyST7XJO7wdQGBKloAAAns;aAAQkIPAAAAAACAWbWAAQ6Ozzrln35KG6DhLXMRYnMjxOmY', '_ga':'GA1.2.894579844.151uus2734989', '_gid':"GA1.2.1055878562.1598994989"
}
for each_url in url_list :
    time.sleep(.1)
    each_url = each_url.replace('http://', 'https://')
    print(each_url)
    response = requests.get(each_url)
    page = response.text
#     print(page)
    soup = BeautifulSoup(page,"html5lib")
    if not soup.find(class_="artwork__label"):
        print("it didn't work")
    else:
        desc = soup.find(class_="artwork__label").text.strip()
        if desc != '' :
            full_list.append((each_url, desc))
        if (c%1000 == 0) :
            print(c)
    c+=1
    #print(page)

https://www.metmuseum.org/art/collection/search/435845
it didn't work


**Test for all webpages using Beautiful Soup**

In [5]:
url_list = ['https://www.metmuseum.org/art/collection/search/437216']
full_list = []
c = 0
for each_url in met_df['Link Resource'] :
    each_url = each_url.replace('http://', 'https://')
    response = requests.get(each_url)
    page = response.text
    print(page)
#     soup = BeautifulSoup(page,"html5lib")
#     desc = soup.find(class_="artwork__label").text.strip()
#     if desc != '' :
#         full_list.append((each_url, desc))
#     if (c%1000 == 0) :
#         print(c)
#     c+=1
    #print(page)
    

<html>
<head>
<META NAME="robots" CONTENT="noindex,nofollow">
<script src="/_Incapsula_Resource?SWJIYLWA=719d34d31c8e3a6e6fffd425f7e032f3">
</script>
<script>
(function() { 
var z="";var b="7472797B766172207868723B76617220743D6E6577204461746528292E67657454696D6528293B766172207374617475733D227374617274223B7661722074696D696E673D6E65772041727261792833293B77696E646F772E6F6E756E6C6F61643D66756E6374696F6E28297B74696D696E675B325D3D22723A222B286E6577204461746528292E67657454696D6528292D74293B646F63756D656E742E637265617465456C656D656E742822696D6722292E7372633D222F5F496E63617073756C615F5265736F757263653F4553324C555243543D363726743D373826643D222B656E636F6465555249436F6D706F6E656E74287374617475732B222028222B74696D696E672E6A6F696E28292B222922297D3B69662877696E646F772E584D4C4874747052657175657374297B7868723D6E657720584D4C48747470526571756573747D656C73657B7868723D6E657720416374697665584F626A65637428224D6963726F736F66742E584D4C4854545022297D7868722E6F6E726561647973746174656368616E67653D66756E6374696F6E

<html>
<head>
<META NAME="robots" CONTENT="noindex,nofollow">
<script src="/_Incapsula_Resource?SWJIYLWA=719d34d31c8e3a6e6fffd425f7e032f3">
</script>
<script>
(function() { 
var z="";var b="7472797B766172207868723B76617220743D6E6577204461746528292E67657454696D6528293B766172207374617475733D227374617274223B7661722074696D696E673D6E65772041727261792833293B77696E646F772E6F6E756E6C6F61643D66756E6374696F6E28297B74696D696E675B325D3D22723A222B286E6577204461746528292E67657454696D6528292D74293B646F63756D656E742E637265617465456C656D656E742822696D6722292E7372633D222F5F496E63617073756C615F5265736F757263653F4553324C555243543D363726743D373826643D222B656E636F6465555249436F6D706F6E656E74287374617475732B222028222B74696D696E672E6A6F696E28292B222922297D3B69662877696E646F772E584D4C4874747052657175657374297B7868723D6E657720584D4C48747470526571756573747D656C73657B7868723D6E657720416374697665584F626A65637428224D6963726F736F66742E584D4C4854545022297D7868722E6F6E726561647973746174656368616E67653D66756E6374696F6E

<html>
<head>
<META NAME="robots" CONTENT="noindex,nofollow">
<script src="/_Incapsula_Resource?SWJIYLWA=719d34d31c8e3a6e6fffd425f7e032f3">
</script>
<script>
(function() { 
var z="";var b="7472797B766172207868723B76617220743D6E6577204461746528292E67657454696D6528293B766172207374617475733D227374617274223B7661722074696D696E673D6E65772041727261792833293B77696E646F772E6F6E756E6C6F61643D66756E6374696F6E28297B74696D696E675B325D3D22723A222B286E6577204461746528292E67657454696D6528292D74293B646F63756D656E742E637265617465456C656D656E742822696D6722292E7372633D222F5F496E63617073756C615F5265736F757263653F4553324C555243543D363726743D373826643D222B656E636F6465555249436F6D706F6E656E74287374617475732B222028222B74696D696E672E6A6F696E28292B222922297D3B69662877696E646F772E584D4C4874747052657175657374297B7868723D6E657720584D4C48747470526571756573747D656C73657B7868723D6E657720416374697665584F626A65637428224D6963726F736F66742E584D4C4854545022297D7868722E6F6E726561647973746174656368616E67653D66756E6374696F6E

<html>
<head>
<META NAME="robots" CONTENT="noindex,nofollow">
<script src="/_Incapsula_Resource?SWJIYLWA=719d34d31c8e3a6e6fffd425f7e032f3">
</script>
<script>
(function() { 
var z="";var b="7472797B766172207868723B76617220743D6E6577204461746528292E67657454696D6528293B766172207374617475733D227374617274223B7661722074696D696E673D6E65772041727261792833293B77696E646F772E6F6E756E6C6F61643D66756E6374696F6E28297B74696D696E675B325D3D22723A222B286E6577204461746528292E67657454696D6528292D74293B646F63756D656E742E637265617465456C656D656E742822696D6722292E7372633D222F5F496E63617073756C615F5265736F757263653F4553324C555243543D363726743D373826643D222B656E636F6465555249436F6D706F6E656E74287374617475732B222028222B74696D696E672E6A6F696E28292B222922297D3B69662877696E646F772E584D4C4874747052657175657374297B7868723D6E657720584D4C48747470526571756573747D656C73657B7868723D6E657720416374697665584F626A65637428224D6963726F736F66742E584D4C4854545022297D7868722E6F6E726561647973746174656368616E67653D66756E6374696F6E

<html>
<head>
<META NAME="robots" CONTENT="noindex,nofollow">
<script src="/_Incapsula_Resource?SWJIYLWA=719d34d31c8e3a6e6fffd425f7e032f3">
</script>
<script>
(function() { 
var z="";var b="7472797B766172207868723B76617220743D6E6577204461746528292E67657454696D6528293B766172207374617475733D227374617274223B7661722074696D696E673D6E65772041727261792833293B77696E646F772E6F6E756E6C6F61643D66756E6374696F6E28297B74696D696E675B325D3D22723A222B286E6577204461746528292E67657454696D6528292D74293B646F63756D656E742E637265617465456C656D656E742822696D6722292E7372633D222F5F496E63617073756C615F5265736F757263653F4553324C555243543D363726743D373826643D222B656E636F6465555249436F6D706F6E656E74287374617475732B222028222B74696D696E672E6A6F696E28292B222922297D3B69662877696E646F772E584D4C4874747052657175657374297B7868723D6E657720584D4C48747470526571756573747D656C73657B7868723D6E657720416374697665584F626A65637428224D6963726F736F66742E584D4C4854545022297D7868722E6F6E726561647973746174656368616E67653D66756E6374696F6E

...

<html style="height:100%"><head><META NAME="ROBOTS" CONTENT="NOINDEX, NOFOLLOW"><meta name="format-detection" content="telephone=no"><meta name="viewport" content="initial-scale=1.0"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><script type="text/javascript" src="/_Incapsula_Resource?SWJIYLWA=719d34d31c8e3a6e6fffd425f7e032f3"></script></head><body style="margin:0px;height:100%"><iframe src="/_Incapsula_Resource?CWUDNSAI=28&xinfo=12-43627526-0%200NNN%20RT%281526953245837%2095%29%20q%280%20-1%20-1%20-1%29%20r%280%20-1%29%20B12%284%2c315%2c0%29%20U18&incident_id=149006090092683049-315667829839177244&edet=12&cinfo=04000000" frameborder=0 width="100%" height="100%" marginheight="0px" marginwidth="0px">Request unsuccessful. Incapsula incident ID: 149006090092683049-315667829839177244</iframe></body></html>
<html style="height:100%"><head><META NAME="ROBOTS" CONTENT="NOINDEX, NOFOLLOW"><meta name="format-detection" content="telephone=no"><meta name="viewport" content="initial

<html style="height:100%"><head><META NAME="ROBOTS" CONTENT="NOINDEX, NOFOLLOW"><meta name="format-detection" content="telephone=no"><meta name="viewport" content="initial-scale=1.0"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><script type="text/javascript" src="/_Incapsula_Resource?SWJIYLWA=719d34d31c8e3a6e6fffd425f7e032f3"></script></head><body style="margin:0px;height:100%"><iframe src="/_Incapsula_Resource?CWUDNSAI=28&xinfo=4-28554897-0%200NNN%20RT%281526953247097%20156%29%20q%280%20-1%20-1%20-1%29%20r%280%20-1%29%20B12%284%2c315%2c0%29%20U18&incident_id=149006090092683049-205848896219719188&edet=12&cinfo=04000000" frameborder=0 width="100%" height="100%" marginheight="0px" marginwidth="0px">Request unsuccessful. Incapsula incident ID: 149006090092683049-205848896219719188</iframe></body></html>
<html style="height:100%"><head><META NAME="ROBOTS" CONTENT="NOINDEX, NOFOLLOW"><meta name="format-detection" content="telephone=no"><meta name="viewport" content="initial

<html style="height:100%"><head><META NAME="ROBOTS" CONTENT="NOINDEX, NOFOLLOW"><meta name="format-detection" content="telephone=no"><meta name="viewport" content="initial-scale=1.0"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><script type="text/javascript" src="/_Incapsula_Resource?SWJIYLWA=719d34d31c8e3a6e6fffd425f7e032f3"></script></head><body style="margin:0px;height:100%"><iframe src="/_Incapsula_Resource?CWUDNSAI=28&xinfo=7-23848968-0%200NNN%20RT%281526953248253%20111%29%20q%280%20-1%20-1%20-1%29%20r%280%20-1%29%20B12%284%2c315%2c0%29%20U18&incident_id=149006090092683049-177173233535362583&edet=12&cinfo=04000000" frameborder=0 width="100%" height="100%" marginheight="0px" marginwidth="0px">Request unsuccessful. Incapsula incident ID: 149006090092683049-177173233535362583</iframe></body></html>
<html style="height:100%"><head><META NAME="ROBOTS" CONTENT="NOINDEX, NOFOLLOW"><meta name="format-detection" content="telephone=no"><meta name="viewport" content="initial

<html style="height:100%"><head><META NAME="ROBOTS" CONTENT="NOINDEX, NOFOLLOW"><meta name="format-detection" content="telephone=no"><meta name="viewport" content="initial-scale=1.0"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><script type="text/javascript" src="/_Incapsula_Resource?SWJIYLWA=719d34d31c8e3a6e6fffd425f7e032f3"></script></head><body style="margin:0px;height:100%"><iframe src="/_Incapsula_Resource?CWUDNSAI=28&xinfo=12-43627899-0%200NNN%20RT%281526953249454%2056%29%20q%280%20-1%20-1%20-1%29%20r%280%20-1%29%20B12%284%2c315%2c0%29%20U18&incident_id=149006090092683049-315670432589358620&edet=12&cinfo=04000000" frameborder=0 width="100%" height="100%" marginheight="0px" marginwidth="0px">Request unsuccessful. Incapsula incident ID: 149006090092683049-315670432589358620</iframe></body></html>
<html style="height:100%"><head><META NAME="ROBOTS" CONTENT="NOINDEX, NOFOLLOW"><meta name="format-detection" content="telephone=no"><meta name="viewport" content="initial

KeyboardInterrupt: 

**Looks like it blocks the Beautiful Soup robot from scraping...**  
**Test using Chrome Webdriver for Mac and Beautiful Soup**

In [3]:
met_df = pd.read_csv('../files/MetURL.csv', header=None, names=['ind', 'Link Resource'])
len(met_df)

466142

In [116]:
#chromedriver = "/Users/monita/ds/metis/metisgh/bootcamp/P4_ArtRecommender/chromedriver"
chromedriver = "../utils/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [67]:
url_list = met_df['Link Resource'][:100]
url_list

0     http://www.metmuseum.org/art/collection/search/1
1     http://www.metmuseum.org/art/collection/search/2
2     http://www.metmuseum.org/art/collection/search/3
3     http://www.metmuseum.org/art/collection/search/4
4     http://www.metmuseum.org/art/collection/search/5
5     http://www.metmuseum.org/art/collection/search/6
6     http://www.metmuseum.org/art/collection/search/7
7     http://www.metmuseum.org/art/collection/search/8
8     http://www.metmuseum.org/art/collection/search/9
9    http://www.metmuseum.org/art/collection/search/10
Name: Link Resource, dtype: object

**Test de todas las URLs usando timeOut para evitar ser expulsados por el Servidor**  
El valor de c debe iniciar en 0 y cambiarse manualmente cada vez que el servidor arroje TimeOutException

In [117]:
full_list=[]
#url_list = ["https://www.metmuseum.org/art/collection/search/437216", "https://www.metmuseum.org/art/collection/search/435844"]
# 486000
c=450001
url_list = met_df['Link Resource'][c-1:]

for each_url in url_list :
    try :
        driver.get(each_url)
    except TimeoutException :
        print("Timed Out " + str(c) + ": " + each_url)
        driver.get(each_url)
        continue
        
    soup = BeautifulSoup(driver.page_source,"html5lib")
    desc = ''
    long_desc = ''
    if soup.find(class_="artwork__label"):
        desc = soup.find(class_="artwork__label").text.strip()
    if soup.find(text=re.compile('Catalogue Entry')):
        long_desc = soup.find(text=re.compile('Catalogue Entry')).find_parent("header").findNextSibling().text.strip()
    if (desc != '' or long_desc != ''):
            full_list.append((each_url, desc, long_desc))
    if (c%500 == 0) :
        print(c)
    c+=1
    time.sleep(.1)

450500
451000
451500
452000
452500
453000
453500
454000
454500
455000
455500
456000
456500
457000
457500
458000
458500
459000
459500
460000
460500
461000
461500
462000
462500
463000
463500
464000
464500
465000
465500
466000


In [118]:
c

466143

In [119]:
len(full_list)

9778

Se van guardando las descripciones en un pickleFile, en el siguiente paso del análisis se cargarán todos consecutivamente

In [120]:
with open('../MetFiles/Met_descriptions_450000-FIN.pkl', 'wb') as f:
    pickle.dump(full_list, f)